In [5]:
from geotext import GeoText
import pandas as pd
import numpy as np
import ast
import csv
import json
import re
import geograpy
import urllib2

In [6]:
def load_json_multiple(segments):
    #chunk = ""
    for segment in segments:
        #chunk += segment
        try:
            yield json.loads(segment)
            #chunk = ""
        except ValueError:
            pass


tweets_json = []

with open('merged.json') as f:
    for parsed_json in load_json_multiple(f):
        tweets_json.append(parsed_json)

In [7]:
text = []
locations = []
for line in tweets_json:
    if line == 420:
        continue
    if 'text' in line:
        text.append(line['text'])
    if 'user' in line:
        if 'location' in line['user']:
            locations.append(line['user']['location'])

In [8]:
#locations

In [9]:
regexV = r'.*vaccine.*'
regexO = r'.*outbreak.*'

vaccine = []
outbreak = []
vaccineLoc = []
outbreakLoc = []

for line in text:
    if re.match(regexV, line.encode('utf-8'), re.IGNORECASE):
        vaccine.append(line.encode('utf-8'))
        temp = locations[text.index(line)]
        if temp != None:
            vaccineLoc.append(temp.encode('utf-8'))
        else:
            vaccineLoc.append(temp)
    if re.match(regexO, line.encode('utf-8'), re.IGNORECASE):
        outbreak.append(line.encode('utf-8'))
        temp = locations[text.index(line)]
        if temp != None:
            outbreakLoc.append(temp.encode('utf-8'))
        else:
            vaccineLoc.append(temp)

In [10]:
from itertools import izip
with open('vac_loc.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(vaccine, vaccineLoc))

In [11]:
with open('break_loc.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(outbreak, outbreakLoc))

In [22]:
Lon = []
Lat = []
empty = []
repeat = []

In [23]:
for line in outbreak:
    temp = GeoText(line)
    if temp.cities or temp.countries:
        if temp.cities:
            for c in temp.cities:
                tmp = c.replace(" ","%20")
                response = urllib2.urlopen("http://nominatim.openstreetmap.org/search?q="+ tmp +"&format=json&addressdetails=1&limit=1")
                data = json.load(response)
                if len(data) != 0:
                    if (data[0]['lat'] and data[0]['lon']):
                        tmplat = data[0]['lat'].encode('utf-8')
                        tmplon = data[0]['lon'].encode('utf-8')
                        if tmplat in Lat and tmplon in Lon:
                            repeat[Lat.index(tmplat)] += 1
                        else:
                            Lat.append(tmplat)
                            Lon.append(tmplon)
                            repeat.append(1)
                    else:
                        empty.append(line)
                else:
                    empty.append(c)
        else:
            for co in temp.countries:
                tmp = co.replace(" ","%20")
                response = urllib2.urlopen("http://nominatim.openstreetmap.org/search?q="+ tmp +"&format=json&addressdetails=1&limit=1")
                data = json.load(response)
                if len(data) != 0:
#                     print("country\n")
#                     print('lat' in data[0])
#                     print("\n")
                    if (data[0]['lat'] and data[0]['lon']):
                        tmplat = data[0]['lat'].encode('utf-8')
                        tmplon = data[0]['lon'].encode('utf-8')
                        if tmplat in Lat and tmplon in Lon:
                            repeat[Lat.index(tmplat)] += 1
                        else:
                            Lat.append(tmplat)
                            Lon.append(tmplon)
                            repeat.append(1)
                    else:
                        empty.append(line)
                else:
                    empty.append(co)

In [24]:
string = ["Mopani", "Vhembe", "Binga", "Limpopo"]

for s in string:
    response = urllib2.urlopen("http://nominatim.openstreetmap.org/search?q="+ s +"&format=json&addressdetails=1&limit=1")
    data = json.load(response)
    if len(data) != 0:
        if (data[0]['lat'] and data[0]['lon']):
            tmplat = data[0]['lat'].encode('utf-8')
            tmplon = data[0]['lon'].encode('utf-8')
            if tmplat in Lat and tmplon in Lon:
                continue
            else:
                Lat.append(tmplat)
                Lon.append(tmplon)
                tmp = 0
                for line in outbreak:
                    if s in line:
                        tmp += 1
                repeat.append(tmp)
        else:
            empty.append(line)
    else:
        empty.append(c)

In [25]:
len(repeat)

7

In [26]:
with open('break_loc_lon_lat_repeat.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(Lat, Lon, repeat))

In [46]:
vacTextLocation = []
for line in vaccine:
    temp = GeoText(line)
    if temp.cities or temp.countries:
        if temp.cities:
            for c in temp.cities:
                vacTextLocation.append(c)
        else:
            for c in temp.countries:
                vacTextLocation.append(c)
len(vacTextLocation)

59

In [47]:
Lon = []
Lat = []
empty = []

In [48]:
from collections import Counter
temp = Counter(vacTextLocation)
for line in temp:
    repeated = temp[line]
    tmp = line.replace(" ","%20")
    response = urllib2.urlopen("http://nominatim.openstreetmap.org/search?q="+ tmp +"&format=json&addressdetails=1&limit=1")
    data = json.load(response)
    if len(data) != 0:
        if (data[0]['lat'] and data[0]['lon']):
            tmplat = data[0]['lat'].encode('utf-8')
            tmplon = data[0]['lon'].encode('utf-8')
            for i in range(0, repeated):
                Lat.append(tmplat)
                Lon.append(tmplon)             
        else:
            empty.append(line)
    else:
        empty.append(line)

In [49]:
with open('vac_text_lon_lat.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(Lat, Lon))

In [50]:
Lon = []
Lat = []
empty = []
repeat = []

In [51]:
vaccineLocWithoutNone = []

for lc in vaccineLoc:
    if lc != None:
        vaccineLocWithoutNone.append(lc)

tmpVacLoc = Counter(vaccineLocWithoutNone)

In [55]:
regexHashtag = r'#(\w+)'

for line in tmpVacLoc:
    if re.match(regexHashtag, line):
        empty.append(line)
        continue
    repeated = tmpVacLoc[line]
    tmp = line.replace(" ","%20")
    response = urllib2.urlopen("http://nominatim.openstreetmap.org/search?q="+ tmp +"&format=json&addressdetails=1&limit=1")
    data = json.load(response)
    if len(data) != 0:
        if (data[0]['lat'] and data[0]['lon']):
            tmplat = data[0]['lat'].encode('utf-8')
            tmplon = data[0]['lon'].encode('utf-8')
            for i in range(0, repeated):
                Lat.append(tmplat)
                Lon.append(tmplon)             
        else:
            empty.append(line)
    else:
        empty.append(line)